In [1]:
import string
s = string.ascii_lowercase


In [1]:
import pandas as pd
import numpy as np

In [21]:
m = MyLineReg()

In [ ]:
class MyLineReg:


    def __init__(self,learning_rate, sgd_sample = None, n_iter=100, weights=None, metric = None, reg:str = None, l1_coef = 0, l2_coef = 0, random_state = 42):

        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = weights
        self.metric = metric
        self.l1_coef = l1_coef
        self.l2_coef = l1_coef
        self.reg = reg
        self.random_state = random_state
        self.sgd_sample = sgd_sample
        
    @staticmethod
    def _l1(l1_coef ,l2_coef ,weights):
        l2_coef = 0
        reg_loss = l1_coef* np.sum(np.abs(weights))
        reg_grad = l1_coef * np.sign(weights)
        return reg_loss, reg_grad
    
    @staticmethod
    def _l2(l1_coef,l2_coef,weights):
        l1_coef = 0
        reg_loss = l2_coef * np.sum(weights**2)
        reg_grad = l2_coef * np.dot(weights,2)
        return reg_loss, reg_grad
    
    @staticmethod
    def _elasticnet(l1_coef, l2_coef, weights):
        reg_loss = l1_coef* np.sum(np.abs(weights)) + l2_coef* np.sum(weights**2)
        reg_grad = l1_coef * np.sign(weights) + l2_coef * (2*weights)
        return reg_loss, reg_grad
    
    def __repr__(self):
        return f"MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}"


    def fit(self, X_df: pd.DataFrame, Y_target: pd.Series, verbose=False):
        random.seed(self.random_state)
        self.X_df = X_df.insert(0, 'w0', 1)
        n_features = X_df.shape[1]
        self.weights = np.ones(n_features)
        reg_loss, reg_grad = 0, 0
        if self.sgd_sample == None:
            self.sgd_sample = X_df.shape[0]
        elif isinstance(self.sgd_sample, float):
            self.sgd_sample = round(X_df.shape[0] * self.sgd_sample)
        for i in range(1,self.n_iter+1):
            if callable(self.learning_rate):
                learning_r = self.learning_rate(i)
            else:
                learning_r = self.learning_rate
            if self.reg:
                reg_loss, reg_grad = getattr(self, '_' + self.reg)(self.l1_coef,self.l2_coef,self.weights)
            sample_rows_idx = random.sample(range(X_df.shape[0]), self.sgd_sample)
            y_pred = np.dot(X_df.iloc[sample_rows_idx], self.weights)          
            MSE = np.mean((Y_target - np.dot(X_df, self.weights))**2) + reg_loss
            gradient = 2/X_df.iloc[sample_rows_idx].shape[0] * np.dot(X_df.iloc[sample_rows_idx].T, (y_pred - Y_target.iloc[sample_rows_idx])) + reg_grad
            self.weights -= learning_r * gradient
            y_pred = self.weights @ X_df.T
            result_metric = self.calculate_metric(Y_target, y_pred)
            if verbose:
                if i%verbose:
                     logging.info(f"{i} | loss: {sum(MSE)}| {self.metric}:{result_metric}")
            
    def calculate_metric(self, y, y_hat):
        if self.metric == 'mae':
            return np.mean(np.abs(y - y_hat))
        elif self.metric == 'mse':
            return np.mean((y - y_hat) ** 2)
        elif self.metric == 'rmse':
            return np.sqrt(np.mean((y - y_hat) ** 2))
        elif self.metric == 'mape':
            return 100 * np.mean(np.abs((y - y_hat) / y))
        elif self.metric == 'r2':
            total_variance = np.sum((y - np.mean(y))**2)
            return 1 - np.sum((y - y_hat)**2) / total_variance
        return None
    
    def get_best_score(self):
        return self.result_metric


    def get_coef(self):
            return self.weights[1:] 
            
    def predict(self, X_df):
        X_df.insert(0, 'w0', 1)
        y_pred = np.dot(X_df, self.weights)
        return y_pred
